# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup
Load the API key

In [5]:
using DotEnv
using HTTP, JSON

In [6]:
DotEnv.load()
api_key = ENV["OPENAI_API_KEY"];

## Helper Function

Documentation to make the HTTP request:
https://platform.openai.com/docs/api-reference/making-requests

In [7]:
using HTTP, JSON

function get_completion(prompt, api_key=api_key, temperature=0)
    url = "https://api.openai.com/v1/chat/completions"
    headers = [
        "Content-Type" => "application/json",
        "Authorization" => "Bearer $api_key"
    ]
    data = Dict(
        "model" => "gpt-3.5-turbo",
        "messages" => [Dict("role" => "user", "content" => prompt)],
        "temperature" => temperature
    )
    body = JSON.json(data)
    response = HTTP.post(url, headers=headers, body=body)
    response_dict = JSON.parse(String(response.body))

    # Extract the message
    message = response_dict["choices"][1]["message"]["content"]

    return message
end

get_completion (generic function with 3 methods)

## Product review text

In [9]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had
additional storage and not too high of a price point.
Got it fast.  The string to our lamp broke during the
transit and the company happily sent over a new one.
Came within a few days as well. It was easy to put
together.  I had a missing part, so I contacted their
support and they very quickly got me the missing piece!
Lumina seems to me to be a great company that cares
about their customers and products!!
""";

## Sentiment (positive/negative)

In [10]:
prompt = """
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.

In [12]:
prompt = """
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

positive

## Identify types of emotions

In [13]:
prompt = """
Identify a list of emotions that the writer of the
following review is expressing. Include no more than
five items in the list. Format your answer as a list of
lower-case words separated by commas.

Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

satisfied, grateful, impressed, content, pleased

## Identify anger

In [14]:
prompt = """
Is the writer of the following review expressing anger?
The review is delimited with triple backticks.
Give your answer as either yes or no.

Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

No

## Extract product and company name from customer reviews

In [15]:
prompt = """
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.
  
Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "Lamp with additional storage",
  "Brand": "Lumina"
}

## Doing multiple tasks at once

In [16]:
prompt = """
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''$lamp_review'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}

## Inferring topics

In [17]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

"In a recent survey conducted by the government, \npublic sector employees were asked to rate their level \nof satisfaction with the department they work at. \nThe results revealed that NASA was the most popular \ndepartment with a satisfaction rating of 95%.\n\nOne NASA employ" ⋯ 628 bytes ⋯ "on had the lowest satisfaction \nrating, with only 45% of employees indicating they were \nsatisfied with their job. The government has pledged to \naddress the concerns raised by employees in the survey and \nwork towards improving job satisfaction across all departments.\n"

Infer 5 topics

In [18]:
prompt = """
Determine five topics that are being discussed in the
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''$story'''
"""
response = get_completion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee satisfaction

In [19]:
split(response, ",")

5-element Vector{SubString{String}}:
 "government survey"
 " job satisfaction"
 " NASA"
 " Social Security Administration"
 " employee satisfaction"

In [20]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

5-element Vector{String}:
 "nasa"
 "local government"
 "engineering"
 "employee satisfaction"
 "federal government"

## Make a news alert for certain topics

In [38]:
prompt = """
Determine whether each item in the following list of
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.

List of topics: $topic_list

Text sample: '''$story'''
"""
response = get_completion(prompt)
print(response)

["nasa": 1, "local government": 0, "engineering": 0, "employee satisfaction": 1, "federal government": 1]

In [39]:
response



"[\"nasa\": 1, \"local government\": 0, \"engineering\": 0, \"employee satisfaction\": 1, \"federal government\": 1]"

In [44]:
# Remove the square brackets and split the response by comma
key_value_pairs = split(response[2:end-1], ", ")

# Create the dictionary using a comprehension
topic_dict = Dict(strip(split(pair, ": ")[1], '\"') => parse(Int, split(pair, ": ")[2]) for pair in key_value_pairs)

# Get whether there is news about Nasa
if topic_dict["nasa"] ≥ 1
    println("ALERT: New Nasa Story")
end


ALERT: New Nasa Story
